In [2]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np


2023-09-13 22:05:31.178567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
import pandas as pd
import matplotlib.pyplot as plt

# I want to load a csv placed in ../logs/log_ear_mar.csv
# with this header timestamp;path_img;ear;mar

# And load into a dataframe using pandas

file_path = "../logs/log_all.csv"

df = pd.read_csv(file_path, sep=";")

In [18]:
df.ren

,20230913215706065744,./frames/fatiga/frame_20230913215706065744.jpg,0,0.1,[]
0,20230913215706306115,./frames/fatiga/frame_20230913215706306115.jpg,0.199979,0.172840,"[array([523, 420]), array([525, 446]), array([..."
1,20230913215706436138,./frames/fatiga/frame_20230913215706436138.jpg,0.202157,0.205173,"[array([521, 413]), array([523, 438]), array([..."
2,20230913215706570371,./frames/fatiga/frame_20230913215706570371.jpg,0.209099,0.199986,"[array([519, 414]), array([521, 440]), array([..."
3,20230913215706710969,./frames/fatiga/frame_20230913215706710969.jpg,0.193166,0.174407,"[array([517, 415]), array([520, 442]), array([..."
4,20230913215706868039,./frames/fatiga/frame_20230913215706868039.jpg,0.223245,0.222205,"[array([520, 407]), array([523, 434]), array([..."
...,...,...,...,...,...
398,20230913220455745914,./frames/no fatiga/frame_20230913220455745914.jpg,0.182399,0.092648,"[array([492, 473]), array([495, 501]), array([..."
399,20230913220455892421,./frames/no fatiga/frame_20230913220455892421.jpg,0.207968,0.059496,"[array([489, 479]), array([493, 508]), array([..."
400,20230913220456011033,./frames/no fatiga/frame_20230913220456011033.jpg,0.228706,0.054321,"[array([493, 477]), array([498, 507]), array([..."
401,20230913220456140380,./frames/no fatiga/frame_20230913220456140380.jpg,0.211603,0.072619,"[array([499, 476]), array([502, 506]), array([..."


In [ ]:
df['category'] = df['path_img'].apply(lambda x: x.split('/')[2])
df

# cast ear and mar to float
df['ear'] = df['ear'].astype(float)
df['mar'] = df['mar'].astype(float)

In [13]:
df['category'] = df['path_img'].apply(lambda x: x.split('/')[2])
df

# cast ear and mar to float
df['ear'] = df['ear'].astype(float)
df['mar'] = df['mar'].astype(float)

KeyError: 'path_img'